# Contra Costa County Prosecution Data

###  Notebook originally written by by Umesh Thillaivasan & Dhruv Madaan
### Adapted by Chris Kaiser-Nyman with assistance from Anderson Lam

# Setup

First, let's make sure this notebook works well in both python 2 and 3, import a few common packages, ensure MatplotLib plots figures inline and prepare a function to save the figures:

In [2]:
import os
cwd = os.getcwd()
cwd

'/Users/chris/Desktop/CCC Data Project'

In [3]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import os
import chardet

#import fuzzywuzzy
#from fuzzywuzzy import fuzz
#from fuzzywuzzy import process

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

print ("Setup Complete.")

Setup Complete.


# Data Wrangling

The court data file received requires a lot of cleaning. Let's first read in the data and take a look at what we are dealing with.

Read in the data into a dataframe called court_df.
IF PULLING THIS PROJECT FROM THE GITHUB REPOSITORY: the necessary file is located in a release here: https://github.com/christopherkn/charge-data/releases/download/v1.1/Contra.Costa.Court.Data.Depersonalized.-.Manually.Cleaned.csv

In [4]:
data_file_location = 'Contra Costa Court Data (Depersonalized) - Manually Cleaned.csv'
print (data_file_location)

Contra Costa Court Data (Depersonalized) - Manually Cleaned.csv


In [5]:
# Skip the first 2 rows as the data file has a multi-row header
court_df = pd.read_csv('Contra Costa Court Data (Depersonalized) - Manually Cleaned.csv', skiprows=2, usecols=range(1,443), skipinitialspace=True, index_col=None, low_memory=False)
court_df.head()

,R,S,99999,CITY,20000101,1,CHARGE 1,S.1,DSP,SPLIT 1 1,...,Unnamed: 433,Unnamed: 434,Unnamed: 435,Unnamed: 436,Unnamed: 437,Unnamed: 438,Unnamed: 439,Unnamed: 440,Unnamed: 441,Unnamed: 442
0,W,M,94550,LIVERMORE,20010517,1,CVC 23152(a),M,G,/0003Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H,M,94804,RICHMOND,20061227,1,PC 12031(a)(1),M,DSM,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,94804,RICHMOND,19980403,1,PC 459-460(b),F,G,/0003Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B,F,94565,PITTSBURG,19990209,1,H&S 1135O(a),F,DSM,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4553000000,MARTINEZ,20110726,1,PC 459/460(b),F,G,/0003Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


What is the shape of our data?

In [6]:
court_df.shape

(276232, 442)

In [7]:
court_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276232 entries, 0 to 276231
Columns: 442 entries, R to Unnamed: 442
dtypes: float64(198), int64(24), object(220)
memory usage: 931.5+ MB


We can see that the orignal file has a shape of 276,232 rows, and 442 columns. We know that this is wrong as the original file only has 296 columns, but we will deal with this later.

## Column Headers

In [8]:
headers = pd.read_csv(data_file_location, skipinitialspace=True, usecols=range(1,443), mangle_dupe_cols=True, header=None, na_filter=False, 
                      nrows=2, index_col=None, low_memory=False)
headers

,1,2,3,4,5,6,7,8,9,10,...,433,434,435,436,437,438,439,440,441,442
0,,,ZIP,,COURT,,,,,JAIL/PROB,...,,,,,,,,,,
1,RACE,GENDER,CODE,CITY,FILE DATE,COUNT,CHARGE CODE 1,SEVERITY,DISPOSTION,1,...,,,,,,,,,,


Since the original data has 2 header rows, we will read those rows in separately, then merge them and rename the court_df column names.

Let's fix the Header names, and the rename our main dataframe.

In [9]:
for index in range(0, headers.shape[1] - 1, 1):
    headers = headers.rename(columns = {headers.columns[index]: index})
headers = headers.rename(columns = {headers.columns[441]: 441})

In [10]:
col_names = []
for i in range(0, headers.shape[1], 1):
    joined_column_name = str(headers[i][0].strip() + " ") + " ".join(str(headers[i][1]).strip().split())
    joined_column_name = " ".join(joined_column_name.split())
    joined_column_name = joined_column_name.lower().title()
    col_names.append(joined_column_name)
col_names

['Race',
 'Gender',
 'Zip Code',
 'City',
 'Court File Date',
 'Count',
 'Charge Code 1',
 'Severity',
 'Dispostion',
 'Jail/Prob 1',
 'Prison 1',
 'Jail/Prob 2',
 'Prison 2',
 'Jail/Prob 3',
 'Prison 3',
 'Jail/Prob 4',
 'Prison 4',
 'Jail/Prob 5',
 'Prison 5',
 'Charge Enhance 1',
 'Charge Enhance 2',
 'Charge Enhance 3',
 'Charge Enhance 4',
 'Charge Enhance 5',
 'Probation Rev Type 1',
 'Probation Rev Type 2',
 'Probation Rev Type 3',
 'Probation Rev Type 4',
 'Probation Rev Type 5',
 'Count',
 'Charge Code 2',
 'Severity',
 'Dispostion',
 'Jail/Prob 1',
 'Prison 1',
 'Jail/Prob 2',
 'Prison 2',
 'Jail/Prob 3',
 'Prison 3',
 'Jail/Prob 4',
 'Prison 4',
 'Jail/Prob 5',
 'Prison 5',
 'Charge Enhance 1',
 'Charge Enhance 2',
 'Charge Enhance 3',
 'Charge Enhance 4',
 'Charge Enhance 5',
 'Probation Rev Type 1',
 'Probation Rev Type 2',
 'Probation Rev Type 3',
 'Probation Rev Type 4',
 'Probation Rev Type 5',
 'Count',
 'Charge Code 3',
 'Severity',
 'Disposition',
 'Jail/Prob 1',
 'P

We can see that there are still some column names that are not perfectly cleaned.  Let's find the indices of these headers.

In [11]:
n = col_names.index("Charge Enhanc E 5")
col_names[n] = "Charge Enhance 5"
n = col_names.index("Charge En Hance 5") 
col_names[n] = "Charge Enhance 5"
n = col_names.index('Incident Date 5 Type')
col_names[n] = 'Incident Date 5'
n = col_names.index('Dispostion')
col_names[n] = 'Disposition'

This brings up a new problem: we have duplicate column names! Let's fix that by adding suffixes to the duplicated column names, but leave the error columns blank.

In [12]:
mylist = col_names
newlist = []
for i, v in enumerate(mylist):
    totalcount = mylist.count(v)
    count = mylist[:i].count(v)
    newlist.append(v + " (" + str(count + 1) + ")" if (totalcount > 1 and v != "") else v)

Now let's fix the headers of our dataframe.

In [13]:
court_df.columns = newlist
court_df.head()

,Race,Gender,Zip Code,City,Court File Date,Count (1),Charge Code 1,Severity (1),Disposition (1),Jail/Prob 1 (1),...,,,,,,,,,,
0,W,M,94550,LIVERMORE,20010517,1,CVC 23152(a),M,G,/0003Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H,M,94804,RICHMOND,20061227,1,PC 12031(a)(1),M,DSM,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,94804,RICHMOND,19980403,1,PC 459-460(b),F,G,/0003Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B,F,94565,PITTSBURG,19990209,1,H&S 1135O(a),F,DSM,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4553000000,MARTINEZ,20110726,1,PC 459/460(b),F,G,/0003Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dropping Bad Columns

We can see that there are empty columns after the last real column 'Restitution'. Let's remove these columns.

In [14]:
col_names.index('Restitution')

295

Now let's drop those extra columns beyond 'Restitution'

In [15]:
restitution_column = col_names.index('Restitution')
end_column = court_df.shape[1]
court_df.drop(court_df.columns[restitution_column + 1:end_column], axis=1, inplace=True)
court_df.head()

,Race,Gender,Zip Code,City,Court File Date,Count (1),Charge Code 1,Severity (1),Disposition (1),Jail/Prob 1 (1),...,Incident Date 4,Incident Agency 5,Incident Date 5,Amount,Act 1,Act 2,Act 3,Act 4,Act 5,Restitution
0,W,M,94550,LIVERMORE,20010517,1,CVC 23152(a),M,G,/0003Y,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,H,M,94804,RICHMOND,20061227,1,PC 12031(a)(1),M,DSM,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,94804,RICHMOND,19980403,1,PC 459-460(b),F,G,/0003Y,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,B,F,94565,PITTSBURG,19990209,1,H&S 1135O(a),F,DSM,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4553000000,MARTINEZ,20110726,1,PC 459/460(b),F,G,/0003Y,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# just how much data did we lose?
print("Columns in original dataset: ", end_column)
print("Columns with NA columns dropped: ", court_df.shape[1])
print("Columns dropped:", end_column - court_df.shape[1])

Columns in original dataset:  442
Columns with NA columns dropped:  296
Columns dropped: 146


# Dealing with Whitespace

Now, let's remove extra whitespace by stripping the dataframe.

In [17]:
court_df = court_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [18]:
court_df.sample(5)

,Race,Gender,Zip Code,City,Court File Date,Count (1),Charge Code 1,Severity (1),Disposition (1),Jail/Prob 1 (1),...,Incident Date 4,Incident Agency 5,Incident Date 5,Amount,Act 1,Act 2,Act 3,Act 4,Act 5,Restitution
142310,H,M,94565,BAY POINT,20150915,1,CVC 23152(a)/23550,M,G,0120D/0005Y,...,0,0,0,0,NaN,SECURITY $40.00 AMT,R/F $150.00 AMT,CCA $30.00 AMT,NaN,VICTIM RESTITUTION TO BE DETERMINED
114042,NaN,NaN,0,PITTSBURG,19850925,1,PC 459,F,CTS,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
15267,B,M,4804000000,RICHMOND,20021119,1,CVC 10851(a),F,CTS,0365D/0003Y,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
229137,NaN,NaN,0,BAY POINT,19980313,1,H&S 11351,F,G,0365D/0002Y,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
186643,B,F,4802000000,RICHMOND,19970318,1,PC 459-460(b),F,G,0001D/0002Y,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


# Dealing with Date Columns

Let's handle and parse dates as they are not in datetime format

In [19]:
court_df['Court File Date'].head()

0    20010517
1    20061227
2    19980403
3    19990209
4    20110726
Name: Court File Date, dtype: int64

In [20]:
court_df['Court File Date'] = pd.to_datetime(court_df['Court File Date'], format = "%Y%m%d", errors='coerce')
court_df['Court File Date'].head()

0   2001-05-17
1   2006-12-27
2   1998-04-03
3   1999-02-09
4   2011-07-26
Name: Court File Date, dtype: datetime64[ns]

Great, it worked! Let's do the same for the Incident Date columns 1 through 5

In [21]:
date_string = 'Incident Date '
for x in range(1,6):
    court_df[date_string + str(x)] = pd.to_datetime(court_df[date_string + str(x)], format = "%Y%m%d", errors='coerce')


# Modify the City values case

Finally, let's fix the City column case.

In [22]:
court_df['City'] = court_df['City'].str.title()

In [23]:
court_df.head()

,Race,Gender,Zip Code,City,Court File Date,Count (1),Charge Code 1,Severity (1),Disposition (1),Jail/Prob 1 (1),...,Incident Date 4,Incident Agency 5,Incident Date 5,Amount,Act 1,Act 2,Act 3,Act 4,Act 5,Restitution
0,W,M,94550,Livermore,2001-05-17,1,CVC 23152(a),M,G,/0003Y,...,NaT,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN
1,H,M,94804,Richmond,2006-12-27,1,PC 12031(a)(1),M,DSM,NaN,...,NaT,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,94804,Richmond,1998-04-03,1,PC 459-460(b),F,G,/0003Y,...,NaT,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN
3,B,F,94565,Pittsburg,1999-02-09,1,H&S 1135O(a),F,DSM,NaN,...,NaT,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4553000000,Martinez,2011-07-26,1,PC 459/460(b),F,G,/0003Y,...,NaT,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN


# Create new features

In [24]:
df = court_df.copy()  # Make a copy of court_df as a checkpoint

### Difference in Dates Feature

Create a new Feature called DIFFERENCE IN DATES which are the number of days between the First INCIDENT DATE and COURT FILE DATE

In [25]:
df['Incident to File Days'] = (df['Court File Date'] - df['Incident Date 1']).dt.days  #in days
df.head()

,Race,Gender,Zip Code,City,Court File Date,Count (1),Charge Code 1,Severity (1),Disposition (1),Jail/Prob 1 (1),...,Incident Agency 5,Incident Date 5,Amount,Act 1,Act 2,Act 3,Act 4,Act 5,Restitution,Incident to File Days
0,W,M,94550,Livermore,2001-05-17,1,CVC 23152(a),M,G,/0003Y,...,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN,60.0
1,H,M,94804,Richmond,2006-12-27,1,PC 12031(a)(1),M,DSM,NaN,...,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN,11.0
2,NaN,NaN,94804,Richmond,1998-04-03,1,PC 459-460(b),F,G,/0003Y,...,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN,26.0
3,B,F,94565,Pittsburg,1999-02-09,1,H&S 1135O(a),F,DSM,NaN,...,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN,820.0
4,NaN,NaN,4553000000,Martinez,2011-07-26,1,PC 459/460(b),F,G,/0003Y,...,0,NaT,0,NaN,NaN,NaN,NaN,NaN,NaN,565.0


### Jail and Probation Time Features

Jail and probation times are squished into one column, and formatted oddly. Lets create a function that will separate them, and transform them into days.

In [26]:
def TimeColumns (dframe, dataCol, JailTimeOut, ProbTimeOut):
    dframe[JailTimeOut], dframe[ProbTimeOut] = dframe[dataCol].str.split('/', 1).str  # Splits into Jail and Probation Time
    dframe[JailTimeOut] = dframe[JailTimeOut].astype(str)
    dframe[ProbTimeOut] = dframe[ProbTimeOut].astype(str)
    # df[df['Jail Time'] == ''] # gives a dataframe where "Jail time" is missing
    # df.at[0,'Jail Time'] = 'nan' # turns those values into "nan"
    # df.at[2,'Jail Time'] = 'nan' # BUT ONLY IF YOU MANUALLY INSERT THE ROW NUMBERS IN!
    # df.at[4,'Jail Time'] = 'nan' # THIS SHOULD BE AUTOMATED
    # would also probably want to double check the 'Probation Time' variables
    # This function converts the Input to number of days in jail
    def DaysInJail (value):
        try:
            last_char = value[-1]
        except IndexError:
            return(0)

        DaysPerWeek = 7.0
        DaysPerMonth = 30.0
        DaysPerYear = 365.0
    
        if last_char == 'D': 
            return (float(value[:-1])) 
        elif last_char == 'W': 
            return (float(float(value[:-1]) * DaysPerWeek))
        elif last_char == 'M':
            return (float(value[:-1]) * DaysPerMonth)
        elif last_char == 'Y':
            return (float(value[:-1]) * DaysPerYear)
        else:
            return (0)  # Returns 0 if Received No Jail Time or field was empty ('nan')
    # now apply this function to the column(s):
    dframe[JailTimeOut] = dframe[JailTimeOut].apply(DaysInJail)
    dframe[ProbTimeOut] = dframe[ProbTimeOut].apply(DaysInJail)
    return dframe

Now apply this function to ALL the Jail/Prob columns.

In [27]:
TimeColumns (df, 'Jail/Prob 1 (1)', 'Jail Time 1 (1)', 'Probation Time 1 (1)')
TimeColumns (df, 'Jail/Prob 2 (1)', 'Jail Time 2 (1)', 'Probation Time 2 (1)')
TimeColumns (df, 'Jail/Prob 3 (1)', 'Jail Time 3 (1)', 'Probation Time 3 (1)')
TimeColumns (df, 'Jail/Prob 4 (1)', 'Jail Time 4 (1)', 'Probation Time 4 (1)')
TimeColumns (df, 'Jail/Prob 5 (1)', 'Jail Time 5 (1)', 'Probation Time 5 (1)')

TimeColumns (df, 'Jail/Prob 1 (2)', 'Jail Time 1 (2)', 'Probation Time 1 (2)')
TimeColumns (df, 'Jail/Prob 2 (2)', 'Jail Time 2 (2)', 'Probation Time 2 (2)')
TimeColumns (df, 'Jail/Prob 3 (2)', 'Jail Time 3 (2)', 'Probation Time 3 (2)')
TimeColumns (df, 'Jail/Prob 4 (2)', 'Jail Time 4 (2)', 'Probation Time 4 (2)')
TimeColumns (df, 'Jail/Prob 5 (2)', 'Jail Time 5 (2)', 'Probation Time 5 (2)')

TimeColumns (df, 'Jail/Prob 1 (3)', 'Jail Time 1 (3)', 'Probation Time 1 (3)')
TimeColumns (df, 'Jail/Prob 2 (3)', 'Jail Time 2 (3)', 'Probation Time 2 (3)')
TimeColumns (df, 'Jail/Prob 3 (3)', 'Jail Time 3 (3)', 'Probation Time 3 (3)')
TimeColumns (df, 'Jail/Prob 4 (3)', 'Jail Time 4 (3)', 'Probation Time 4 (3)')
TimeColumns (df, 'Jail/Prob 5 (3)', 'Jail Time 5 (3)', 'Probation Time 5 (3)')

TimeColumns (df, 'Jail/Prob 1 (4)', 'Jail Time 1 (4)', 'Probation Time 1 (4)')
TimeColumns (df, 'Jail/Prob 2 (4)', 'Jail Time 2 (4)', 'Probation Time 2 (4)')
TimeColumns (df, 'Jail/Prob 3 (4)', 'Jail Time 3 (4)', 'Probation Time 3 (4)')
TimeColumns (df, 'Jail/Prob 4 (4)', 'Jail Time 4 (4)', 'Probation Time 4 (4)')
TimeColumns (df, 'Jail/Prob 5 (4)', 'Jail Time 5 (4)', 'Probation Time 5 (4)')

TimeColumns (df, 'Jail/Prob 1 (5)', 'Jail Time 1 (4)', 'Probation Time 1 (5)')
TimeColumns (df, 'Jail/Prob 2 (5)', 'Jail Time 2 (4)', 'Probation Time 2 (5)')
TimeColumns (df, 'Jail/Prob 3 (5)', 'Jail Time 3 (4)', 'Probation Time 3 (5)')
TimeColumns (df, 'Jail/Prob 4 (5)', 'Jail Time 4 (4)', 'Probation Time 4 (5)')
TimeColumns (df, 'Jail/Prob 5 (5)', 'Jail Time 5 (4)', 'Probation Time 5 (5)')

,Race,Gender,Zip Code,City,Court File Date,Count (1),Charge Code 1,Severity (1),Disposition (1),Jail/Prob 1 (1),...,Probation Time 3 (4),Jail Time 4 (4),Probation Time 4 (4),Jail Time 5 (4),Probation Time 5 (4),Probation Time 1 (5),Probation Time 2 (5),Probation Time 3 (5),Probation Time 4 (5),Probation Time 5 (5)
0,W,M,94550,Livermore,2001-05-17,1,CVC 23152(a),M,G,/0003Y,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,H,M,94804,Richmond,2006-12-27,1,PC 12031(a)(1),M,DSM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,94804,Richmond,1998-04-03,1,PC 459-460(b),F,G,/0003Y,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,F,94565,Pittsburg,1999-02-09,1,H&S 1135O(a),F,DSM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,4553000000,Martinez,2011-07-26,1,PC 459/460(b),F,G,/0003Y,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,H,M,93030,Oxnard,2003-05-07,1,PC 12020(a)(1),M,DSM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,W,F,94520,Concord,1996-04-26,2,H&S 11377(a),F,DSM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,W,F,94520,Concord,1996-04-26,2,H&S 11377(a),F,HTA,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,W,F,94597,Walnut Creek,2002-07-18,1,H&S 11377(a),M,G,0120D/0003Y,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,NaN,NaN,0,Richmond,1985-05-10,1,PC 12021,F,DSM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df.sample(5)

,Race,Gender,Zip Code,City,Court File Date,Count (1),Charge Code 1,Severity (1),Disposition (1),Jail/Prob 1 (1),...,Probation Time 3 (4),Jail Time 4 (4),Probation Time 4 (4),Jail Time 5 (4),Probation Time 5 (4),Probation Time 1 (5),Probation Time 2 (5),Probation Time 3 (5),Probation Time 4 (5),Probation Time 5 (5)
56135,W,M,94509,Antioch,2014-01-15,1,H&S 11377(a),M,G,0150D/0002Y,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229465,O,F,94509,Antioch,2014-03-20,1,PC 459/460(b),F,DSM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96640,NaN,P,0,NaN,1992-05-11,0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90395,B,F,94806,San Pablo,2015-02-24,1,CVC 2800.2(a),F,G,0015D/0003Y,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
230132,H,M,94511,Bethel Island,2003-10-06,1,PC 459/460(a),F,DSM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Let's Fix the Zip Codes

In [29]:
def fix_zipcode(zipcode):
    if (zipcode > 99999) & (zipcode <= 1e6): 
        return (zipcode / 1e1)
    elif (zipcode > 1e6) & (zipcode <= 1e7): 
        return (zipcode / 1e2)
    elif (zipcode > 1e7) & (zipcode <= 1e8): 
        return (zipcode / 1e3)
    elif (zipcode > 1e8) & (zipcode <= 1e9): 
        return (zipcode / 1e4)
    elif (zipcode > 1e9) & (zipcode <= 1e10): 
        return (zipcode / 1e5)
    else: 
        return zipcode

In [30]:
df["Zip Code"] = df["Zip Code"].apply(fix_zipcode)

Write this dataframe to a csv, keeping only records from November 1st 2014 and onwards

In [31]:
df[(df['Court File Date'] >= '2014-11-01')].to_csv('aclu_Nov_2014.csv', encoding='utf-8', index=True)

This file is used in the R markdown file for further processing